In [1]:
from pyspark.sql import SparkSession

# get spark session
spark = SparkSession\
        .builder\
        .getOrCreate()

# import packages
import sys
import pandas as pd
import pyspark.pandas as ps
import numpy as np
from timeit import timeit
import gc
import pyspark.sql.functions as sf
from pyspark.context import SparkContext
from pyspark.sql.functions import array_max
from pyspark.ml.feature import StringIndexer, IndexToString
from pyspark.pandas.config import set_option, reset_option
from dateutil.relativedelta import relativedelta
from pyspark.sql.types import (
    DateType, DoubleType, FloatType, IntegerType, StringType, StructField, StructType
)

# import self-defined module
import module.time_split_cv as time_split_cv

# set conf
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", True)
ps.set_option("compute.default_index_type", "distributed")
set_option("compute.ops_on_diff_frames", True)

In [2]:
from pyspark import SparkContext

sc = SparkContext.getOrCreate()
sc.defaultParallelism

88

In [3]:
spark.conf.get("spark.sql.shuffle.partitions")

'800'

In [ ]:
# # get argument
# TRAIN_PERIOD = int(sys.argv[1])
# print(f"\n\n \
# 	==================================================\n \
# 	========= TRAIN_PERIOD = {TRAIN_PERIOD} =========\n \
# 	==================================================\n\n")

In [4]:
TRAIN_PERIOD = 60

In [5]:
# def main(TRAIN_PERIOD):
# print(f"\n\n \
#     ==================================================\n \
#     ================== 1. Load data ==================\n \
#     ==================================================\n\n")
tran_ps = ps.read_parquet('/user/HM_parquet/transactions_train.parquet').drop(['price', 'sales_channel_id'], axis=1)

tran_ps.set_index('t_dat',inplace=True)
tran_ps['start_test'] = ''
tran_ps['split_id'] = ''

# print(f"\n\n \
#     ==================================================\n \
#     ======= 2. Split and concat data by time =========\n \
#     ==================================================\n\n")
split_data, split_id = time_split_cv.split(tran_ps,train_period=TRAIN_PERIOD,test_period=7,stride=30,show_progress=False)
split_data.reset_index(inplace=True)
del tran_ps


# print(f"\n\n \
#     ==================================================\n \
#     ===== 3.Parameter grid table cross split_id ======\n \
#     ==================================================\n\n")
para_cross_split = time_split_cv.make_para_cross_split_table(split_id)

# print(f"\n\n \
#     ==================================================\n \
#     ===== 4. Join split_data parameter grid table ====\n \
#     ==================================================\n\n")

join_data = split_data.join(para_cross_split.set_index('split_id'), on='split_id')
# join_data.set_index('t_dat',inplace=True)
join_data['train_period'] = TRAIN_PERIOD

del split_data, para_cross_split
df = join_data.to_spark()
del join_data

print(f"\n\n \
    ==================================================\n \
    ===== df.rdd.getNumPartitions() = {df.rdd.getNumPartitions()} ===========\n \
    ==================================================\n\n")


gc.collect()

print(f"\n\n \
    ==================================================\n \
    ===== 5. Train SVD model by pandas_UDF ===========\n \
    ==================================================\n\n")
schema = StructType(
    [
        StructField("train_period", IntegerType(), True),
        StructField('start_test', DateType(),True),
        StructField("n_factors", IntegerType(), True),
        StructField("n_epochs", IntegerType(), True),
        StructField('reg_all', FloatType(),True),
        StructField('rmse', FloatType(),True),
        StructField('map12', FloatType(),True),
     ]
)

results = df.groupby('group_id').applyInPandas(time_split_cv.time_split_hyperparameter_search, schema)
del df
gc.collect()

print(f"\n\n \
    ==================================================\n \
    ===== results.rdd.getNumPartitions() = {results.rdd.getNumPartitions()} ===========\n \
    ==================================================\n\n")


print(f"\n\n \
    ==================================================\n \
    ===== 6. Save result table to parquet ============\n \
    ==================================================\n\n")
results.coalesce(1).write.parquet(f'/user/HM_parquet/SVD_model/params/para{TRAIN_PERIOD}.parquet',mode='overwrite')

print(f"\n\n \
    ==================================================\n \
    =====================  存檔完成 ===================\n \
    ==================================================\n\n")





     ===== df.rdd.getNumPartitions() = 23 ===========




     ===== 5. Train SVD model by pandas_UDF ===========






     ===== results.rdd.getNumPartitions() = 364 ===========




     ===== 6. Save result table to parquet ============




2022-04-10 17:35:46,301 WARN scheduler.TaskSetManager: Lost task 0.0 in stage 144.0 (TID 14287) (bdse75.example.com executor 5): org.apache.spark.SparkException: Task failed while writing rows.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.taskFailedWhileWritingRowsError(QueryExecutionErrors.scala:500)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.executeTask(FileFormatWriter.scala:321)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.$anonfun$write$16(FileFormatWriter.scala:229)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:90)
	at org.apache.spark.scheduler.Task.run(Task.scala:131)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$3(Executor.scala:506)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1462)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:509)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadP

Py4JJavaError: An error occurred while calling o4916.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:496)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:251)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:110)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$5(SQLExecution.scala:103)
	at o

In [6]:
# results.coalesce(1).write.parquet(f'/user/HM_parquet/SVD_model/params/para{TRAIN_PERIOD}.parquet',mode='overwrite')
results.write.parquet(f'/user/HM_parquet/SVD_model/params/para{TRAIN_PERIOD}.parquet',mode='overwrite')

2022-04-10 19:41:00,604 WARN scheduler.TaskSetManager: Lost task 21.0 in stage 162.0 (TID 17350) (bdse137.example.com executor 7): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/hdpfs/nm-local-dir/usercache/hadoop/appcache/application_1649580732736_0001/container_e51_1649580732736_0001_01_000008/pyspark.zip/pyspark/worker.py", line 603, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
  File "/hdpfs/nm-local-dir/usercache/hadoop/appcache/application_1649580732736_0001/container_e51_1649580732736_0001_01_000008/pyspark.zip/pyspark/worker.py", line 423, in read_udfs
    arg_offsets, f = read_single_udf(pickleSer, infile, eval_type, runner_conf, udf_index=0)
  File "/hdpfs/nm-local-dir/usercache/hadoop/appcache/application_1649580732736_0001/container_e51_1649580732736_0001_01_000008/pyspark.zip/pyspark/worker.py", line 251, in read_single_udf
    f, return_type = read_command(pickleSer, infile)
  File

Py4JJavaError: An error occurred while calling o4925.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:496)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:251)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:110)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$5(SQLExecution.scala:103)
	at o

2022-04-10 19:41:10,187 WARN scheduler.TaskSetManager: Lost task 26.0 in stage 162.0 (TID 17337) (bdse90.example.com executor 6): TaskKilled (Stage cancelled)
2022-04-10 19:41:10,232 WARN scheduler.TaskSetManager: Lost task 3.0 in stage 162.0 (TID 17343) (bdse92.example.com executor 1): TaskKilled (Stage cancelled)
2022-04-10 19:41:10,232 WARN scheduler.TaskSetManager: Lost task 27.0 in stage 162.0 (TID 17354) (bdse92.example.com executor 1): TaskKilled (Stage cancelled)
2022-04-10 19:41:10,269 WARN scheduler.TaskSetManager: Lost task 20.0 in stage 162.0 (TID 17355) (bdse93.example.com executor 4): TaskKilled (Stage cancelled)
2022-04-10 19:41:10,270 WARN scheduler.TaskSetManager: Lost task 16.0 in stage 162.0 (TID 17344) (bdse93.example.com executor 4): TaskKilled (Stage cancelled)
2022-04-10 19:41:10,315 WARN scheduler.TaskSetManager: Lost task 24.1 in stage 162.0 (TID 17372) (bdse106.example.com executor 2): TaskKilled (Stage cancelled)
2022-04-10 19:41:10,317 WARN scheduler.TaskSet

In [ ]:
results.head(5)

In [13]:
results.count()

2022-04-10 16:13:04,316 WARN cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 28 for reason Container from a bad node: container_e50_1649562268781_0008_01_000001 on host: bdse93.example.com. Exit status: 143. Diagnostics: [2022-04-10 16:13:00.889]Container killed on request. Exit code is 143
[2022-04-10 16:13:01.144]Container exited with a non-zero exit code 143. 
[2022-04-10 16:13:01.144]Killed by external signal
.
2022-04-10 16:13:04,316 ERROR cluster.YarnScheduler: Lost executor 28 on bdse93.example.com: Container from a bad node: container_e50_1649562268781_0008_01_000001 on host: bdse93.example.com. Exit status: 143. Diagnostics: [2022-04-10 16:13:00.889]Container killed on request. Exit code is 143
[2022-04-10 16:13:01.144]Container exited with a non-zero exit code 143. 
[2022-04-10 16:13:01.144]Killed by external signal
.
2022-04-10 16:13:04,317 WARN scheduler.TaskSetManager: Lost task 8.0 in stage 344.0 (TID 40163) (bdse93.example.com exe

KeyboardInterrupt: 

In [ ]:
# t = timeit('main()', setup='gc.enable(); TRAIN_PERIOD = 60', number=1)
# print(f"\n\n \
# ==================================================\n \
# ================== 執行時長: {t} ==================\n \
# ==================================================\n\n")

In [4]:
main(60)



         ===== df.rdd.getNumPartitions() = 23 ===========




         ===== 5. Train SVD model by pandas_UDF ===========






         ===== results.rdd.getNumPartitions() = 178 ===========




         ===== 6. Save result table to parquet ============




2022-04-10 12:40:14,368 ERROR cluster.YarnScheduler: Lost executor 6 on bdse92.example.com: Container from a bad node: container_e46_1649562268781_0008_01_000007 on host: bdse92.example.com. Exit status: 137. Diagnostics: [2022-04-10 12:39:45.039]Container killed on request. Exit code is 137
[2022-04-10 12:39:54.597]Container exited with a non-zero exit code 137. 
[2022-04-10 12:39:55.231]Killed by external signal
.
2022-04-10 12:40:14,368 WARN cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 6 for reason Container from a bad node: container_e46_1649562268781_0008_01_000007 on host: bdse92.example.com. Exit status: 137. Diagnostics: [2022-04-10 12:39:45.039]Container killed on request. Exit code is 137
[2022-04-10 12:39:54.597]Container exited with a non-zero exit code 137. 
[2022-04-10 12:39:55.231]Killed by external signal
.
2022-04-10 12:40:14,372 WARN scheduler.TaskSetManager: Lost task 4.0 in stage 144.0 (TID 14290) (bdse92.example.com execu

Py4JJavaError: An error occurred while calling o4917.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:496)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:251)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:110)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$5(SQLExecution.scala:103)
	at o

2022-04-10 13:27:14,442 WARN scheduler.TaskSetManager: Lost task 0.0 in stage 144.2 (TID 15429) (bdse106.example.com executor 1): org.apache.spark.SparkException: Task failed while writing rows.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.taskFailedWhileWritingRowsError(QueryExecutionErrors.scala:500)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.executeTask(FileFormatWriter.scala:321)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.$anonfun$write$16(FileFormatWriter.scala:229)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:90)
	at org.apache.spark.scheduler.Task.run(Task.scala:131)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$3(Executor.scala:506)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1462)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:509)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.Thread

In [ ]:
main(90)

In [ ]:
main(180)

In [ ]:
main(270)

In [ ]:
main(360)